# Data Pattern: Broken's Angle
## Monstrous Data: Principle Feature Focus

## Abstract
> It is known that when AC is high, ATT is low for any given CR, but both AC and ATT track up with increasing CR. This forms a triad called Broken's Angle. It's broken in the sense that half of it is symmetric and half of it is anti-symmetric.

> **Hypothesis:** It is possible to produce a tighter distribution of data by combining two or more counter-variant features. That is, so long as they are also individually correlated with a control feature - like in the case (AC and ATT) vs CR. In other words - if a Broken's Angle exists can it be used to produce better prediction models than either feature by itself?

> **Null Hypothesis:** No, you can not cancel noise in one channel with the noise of another.

### Data Source
> Data: https://sharpdesigndigital.com/large_monster_set.csv. 

- Stretch Goal: Dataset Created with MonsterFactory.py - based on [Fortuna: Generative Modeling Software](https://pypi.org/project/Fortuna/) by Robert Sharp.

- The D&D Property contained herein is used with permission under the GPL. Game system and monster details are the property of WoTC, Hasbro.

### Broken's Angle
- AC & ATT: When one goes high the other goes low.
- CR: When CR goes up, everything goes up.
<br>

![alt text](https://sharpdesigndigital.com/wp-content/uploads/2020/02/att_ac_cr.jpg)


## Import

In [1]:
import pandas as pd
import numpy as np
import altair as alt
from scipy.stats import ttest_ind
alt.renderers.enable('colab')

RendererRegistry.enable('colab')

## Data Source: Robert Sharp's Monster Factory

In [0]:
monsters = pd.read_csv("https://sharpdesigndigital.com/large_monster_set.csv")

## Monster Info

##### Monster Stat Definitions:
- CR: Challenge Rating
- AC: Armor Class
- HP: Hit Points
- ATT: Attack Power
- DAM: Typical Damage
- Tier: CR Grouping


**Stretch Goal - Dirty Data:** In rulebook - the lowest CR range is divided fractionally dispite having a linear relationship. A simple mapping is used to control this discrepancy, see below.

```
 CR : Mapping
  0 : CR -3
1/8 : CR -2
1/4 : CR -1
1/2 : CR  0

CR range 1-30 is not mapped.
```

In [3]:
monsters.head()

,Monster Name,CR,AC,HP,ATT,DAM,Tier
0,Crawling Claw,-2,9,16,1,2,0
1,Quasit,-2,9,20,1,2,0
2,Flameskull,8,16,179,7,53,2
3,Umber Hulk,2,12,96,3,19,1
4,Gelatinous Spores,-1,10,43,2,4,0


In [4]:
monsters.shape

(1000000, 7)

In [5]:
num_monsters = len(monsters)
unique_monsters = len(monsters['Monster Name'].value_counts())
print(f"Total number of monsters: {num_monsters}")
print(f"Number of unique monster names: {unique_monsters}")

Total number of monsters: 1000000
Number of unique monster names: 3095


**Stretch Goal - Large Data:** A sample size of 5000 in the CR range of 1-20 will be used for visualizations.

## Monster AC and ATT

In [0]:
subset_monsters = monsters.sample(5000)

In [7]:
ac = alt.Chart(
    subset_monsters, 
    title="Monster AC by CR"
).mark_circle(size=75).encode(
    x='CR:O',
    y='AC:Q',
    color='Tier:N',
    tooltip=list(monsters.columns),
).properties(width=580, height=240)

att = alt.Chart(
    subset_monsters, 
    title="Monster ATT by CR"
).mark_circle(size=75).encode(
    x='CR:O',
    y='ATT:Q',
    color='Tier:N',
    tooltip=list(monsters.columns),
).properties(width=580, height=240)
ac.save("ac.html")
att.save("att.html")
ac_and_att = ac & att
ac_and_att.save("ac_and_att.html")
ac_and_att

alt.VConcatChart(...)

## Monster AC plus ATT

In [0]:
# Folding Broken's Angle
subset_monsters['AC + ATT'] = subset_monsters['AC'] + subset_monsters['ATT']

In [12]:
ac_att = alt.Chart(
    subset_monsters, 
    title="Monster AC + ATT by CR"
).mark_circle(size=150).encode(
    x='CR:O',
    y='AC + ATT:Q',
    color='Tier:N',
    tooltip=list(monsters.columns),
).properties(width=580, height=400)
ac_att.save("ac_att.html")
ac_att

alt.Chart(...)

- The graph above represents the primary point or crux of a Broken Angle, it's like the tip of the arrow.

## Scaling Smears the Data

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_ac = scaler.fit_transform(subset_monsters[['AC']])
scaled_att = scaler.fit_transform(subset_monsters[['ATT']])
subset_monsters['Pre Scaled'] = scaled_ac + scaled_att
subset_monsters

,Monster Name,CR,AC,HP,ATT,DAM,Tier,AC + ATT,Pre Scaled
319123,"Vampire Spawn, Halfling",-3,8,4,0,1,0,8,-3.408568
125458,Jackalope,-1,10,43,2,4,0,12,-2.310398
615913,Wyvern,9,17,199,6,61,2,23,0.679323
95653,Gnomish War Tank,18,16,347,13,100,4,29,2.407235
769819,Ghoul,1,10,78,4,9,1,14,-1.741149
...,...,...,...,...,...,...,...,...,...
853232,Blue Dragon Hatchling,0,11,59,3,7,0,14,-1.761313
755982,Bafu,2,12,93,3,18,1,15,-1.496853
562703,Young Red Dragon,1,11,76,3,12,1,14,-1.761313
673425,Basilisk,9,16,200,7,61,2,23,0.699487


In [13]:
ac_att_blur = alt.Chart(
    subset_monsters, 
    title="AC+ATT Scaled"
).mark_circle(size=150).encode(
    x='CR:O',
    y='Pre Scaled:Q',
    color='Tier:N',
    tooltip=list(monsters.columns),
).properties(width=580, height=400)
ac_att.save("ac_att.html")
ac_att | ac_att_blur

alt.HConcatChart(...)

# Summary
This is amazing! By combining features with simple addition, the data becomes much more focused and therefor better for predictive models. Now I can predict the CR of a monster based on its stats with +/- 1 CR accuracy. Either feature individually was only good for +/- 6 CR predictions at best.

## Folding Broken's Angle:

In [0]:
monsters['AC + ATT'] = monsters['AC'] + monsters['ATT']

### Stratified Tier Variance: Sample Data - Full CR Range


In [15]:
ac_vs_att = alt.Chart(
    monsters.sample(5000), 
    title="Broken Angle",
).mark_circle(size=150).encode(
    x='AC:O',
    y='ATT:Q',
    color='Tier:N',
    tooltip=list(monsters.columns),
).properties(width=580, height=460)
ac_vs_att.save("ac_vs_att.html")
ac_vs_att

alt.Chart(...)